In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 24.1 MB/s eta 0:00:00


In [ ]:
!pip install MissForest

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/Dacon/datasets/기업 성공 확률 예측 해커톤: 미래의 성공기업을 발굴하라!_Dataset/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Dacon/datasets/기업 성공 확률 예측 해커톤: 미래의 성공기업을 발굴하라!_Dataset/test.csv")
sample_sub = pd.read_csv("/content/drive/MyDrive/Dacon/datasets/기업 성공 확률 예측 해커톤: 미래의 성공기업을 발굴하라!_Dataset/sample_submission.csv")

In [ ]:
train_df.head()

,ID,설립연도,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),성공확률
0,TRAIN_0000,2009,CT005,이커머스,Series A,4126.0,No,No,56.0,3365.0,4764.0,4.71,NaN,0.3
1,TRAIN_0001,2023,CT006,핀테크,Seed,4167.0,Yes,No,80.0,4069.0,279.0,1.00,2500-3500,0.8
2,TRAIN_0002,2018,CT007,기술,Series A,3132.0,Yes,Yes,54.0,6453.0,12141.0,4.00,3500-4500,0.5
3,TRAIN_0003,2016,CT006,NaN,Seed,3245.0,Yes,Yes,NaN,665.0,10547.0,2.97,NaN,0.7
4,TRAIN_0004,2020,CT002,에듀테크,Seed,1969.0,No,Yes,94.0,829.0,9810.0,1.00,1500-2500,0.1


In [ ]:
test_df.head()

,ID,설립연도,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원)
0,TEST_0000,2002,CT010,핀테크,Series C,3261.0,No,Yes,45.0,5021.0,6680.0,2.00,1500-2500
1,TEST_0001,2020,CT001,푸드테크,Series C,3707.0,Yes,No,70.0,1601.0,4654.0,4.20,NaN
2,TEST_0002,2014,CT006,에듀테크,IPO,236.0,Yes,Yes,89.0,4709.0,9289.0,1.00,6000이상
3,TEST_0003,2003,CT001,에너지,Seed,637.0,Yes,Yes,17.0,2145.0,7005.0,5.00,1500-2500
4,TEST_0004,2006,CT010,핀테크,Seed,4922.0,Yes,No,68.0,4995.0,7593.0,4.36,NaN


In [ ]:
sample_sub.head()

,ID,성공확률
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0


In [ ]:
train_df.shape

(4376, 14)

In [ ]:
test_df.shape

(1755, 13)

In [ ]:
train_df.isnull().sum()

,0
ID,0
설립연도,0
국가,0
분야,857
투자단계,0
직원 수,174
인수여부,0
상장여부,0
고객수(백만명),1320
총 투자금(억원),0


In [ ]:
test_df.isnull().sum()

,0
ID,0
설립연도,0
국가,0
분야,354
투자단계,0
직원 수,76
인수여부,0
상장여부,0
고객수(백만명),547
총 투자금(억원),0


---

In [ ]:
# # ✅ train_df의 결측치가 포함된 row 삭제
# train_df_dropna_rows = train_df.dropna()

In [ ]:
# train_df_dropna_rows.shape

In [ ]:
# train_df_dropna_rows.head()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from missforest import MissForest
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# ✅ 1. 복사본 생성
train = train_df.copy()
test = test_df.copy()

# ✅ 2. 타겟 및 제외할 컬럼
target_col = '성공확률'
exclude_cols = ['ID', target_col]

# ✅ 3. 결측치 있는 컬럼과 범주형 변수 정의
missing_cols = ['분야', '직원 수', '고객수(백만명)', '기업가치(백억원)']
categorical_cols = ['국가', '분야', '투자단계', '인수여부', '상장여부', '기업가치(백억원)']

# ✅ 4. 보간 대상 컬럼만 추출
train_features = train.drop(columns=exclude_cols).copy()
test_features = test.drop(columns=['ID']).copy()

# ✅ 5. Label Encoding (train + test 모두 같은 방식으로)
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    full_series = pd.concat([train_features[col], test_features[col]], axis=0)
    le.fit(full_series.dropna())

    train_features[col] = train_features[col].map(lambda x: le.transform([x])[0] if pd.notnull(x) else np.nan)
    test_features[col] = test_features[col].map(lambda x: le.transform([x])[0] if pd.notnull(x) else np.nan)

    label_encoders[col] = le

# ✅ 6. MissForest 초기화
mf = MissForest(
    categorical=categorical_cols,
    clf=RandomForestClassifier(n_jobs=-1),
    rgr=RandomForestRegressor(n_jobs=-1),
)

# ✅ 7. MissForest fit() + transform() (최신 방식)
mf.fit(train_features)
train_imputed = mf.transform(train_features)
test_imputed = mf.transform(test_features)

# ✅ 8. Label Decoding (숫자 → 원래 값 복원)
for col in categorical_cols:
    le = label_encoders[col]
    train_imputed[col] = train_imputed[col].map(lambda x: le.inverse_transform([int(x)])[0])
    test_imputed[col] = test_imputed[col].map(lambda x: le.inverse_transform([int(x)])[0])

# ✅ 9. 결측치가 보간된 결과를 원래 데이터프레임에 반영
train.update(train_imputed)
test.update(test_imputed)



/usr/local/lib/python3.11/dist-packages/missforest/missforest.py:333: UserWarning: Label encoding is no longer performed by default. Users will have to perform categorical features encoding by themselves.
  warnings.warn("Label encoding is no longer performed by default. "
100%|██████████| 5/5 [00:23<00:00,  4.61s/it]
/usr/local/lib/python3.11/dist-packages/missforest/missforest.py:490: UserWarning: Label encoding is no longer performed by default. Users will have to perform categorical features encoding by themselves.
  warnings.warn("Label encoding is no longer performed by default. "
/usr/local/lib/python3.11/dist-packages/missforest/missforest.py:494: UserWarning: In version 4.2.3, estimator fitting process is moved to `fit` method. `MissForest` will now imputes unseen missing values with fitted estimators with `transform` method. To retain the old behaviour, use `fit_transform` to fit the whole unseen data instead.
  warnings.warn(f"In version {VERSION}, estimator fitting process 

In [ ]:
train.head()

,ID,설립연도,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),성공확률
0,TRAIN_0000,2009,CT005,이커머스,Series A,4126.0,No,No,56.000000,3365.0,4764.0,4.71,4500-6000,0.3
1,TRAIN_0001,2023,CT006,핀테크,Seed,4167.0,Yes,No,80.000000,4069.0,279.0,1.00,2500-3500,0.8
2,TRAIN_0002,2018,CT007,기술,Series A,3132.0,Yes,Yes,54.000000,6453.0,12141.0,4.00,3500-4500,0.5
3,TRAIN_0003,2016,CT006,에너지,Seed,3245.0,Yes,Yes,45.818713,665.0,10547.0,2.97,4500-6000,0.7
4,TRAIN_0004,2020,CT002,에듀테크,Seed,1969.0,No,Yes,94.000000,829.0,9810.0,1.00,1500-2500,0.1


In [ ]:
test.head()

,ID,설립연도,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원)
0,TEST_0000,2002,CT010,핀테크,Series C,3261.0,No,Yes,45.0,5021.0,6680.0,2.00,1500-2500
1,TEST_0001,2020,CT001,푸드테크,Series C,3707.0,Yes,No,70.0,1601.0,4654.0,4.20,4500-6000
2,TEST_0002,2014,CT006,에듀테크,IPO,236.0,Yes,Yes,89.0,4709.0,9289.0,1.00,6000이상
3,TEST_0003,2003,CT001,에너지,Seed,637.0,Yes,Yes,17.0,2145.0,7005.0,5.00,1500-2500
4,TEST_0004,2006,CT010,핀테크,Seed,4922.0,Yes,No,68.0,4995.0,7593.0,4.36,4500-6000


In [ ]:
train.isnull().sum()

,0
ID,0
설립연도,0
국가,0
분야,0
투자단계,0
직원 수,0
인수여부,0
상장여부,0
고객수(백만명),0
총 투자금(억원),0


In [ ]:
test.isnull().sum()

,0
ID,0
설립연도,0
국가,0
분야,0
투자단계,0
직원 수,0
인수여부,0
상장여부,0
고객수(백만명),0
총 투자금(억원),0


In [ ]:
# train_df_imputed = train.copy()


train_df_dropna_rows = train_df.dropna().reset_index(drop=True)
train_df_imputed = train_df_dropna_rows.copy()


In [ ]:
test_df_imputed = test.copy()

NameError: name 'test' is not defined

---

In [ ]:
current_year = 2025

In [ ]:
# ✅ '기업나이' 컬럼 생성 (현재년도 - 설립연도)
train_df_imputed['기업나이'] = current_year - train_df_imputed['설립연도']

# ✅ 기존 '설립연도' 컬럼 제거
train_df_imputed = train_df_imputed.drop(columns=['설립연도'])



In [ ]:
train_df_imputed.head()

,ID,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),성공확률,기업나이
0,TRAIN_0001,CT006,핀테크,Seed,4167.0,Yes,No,80.0,4069.0,279.0,1.0,2500-3500,0.8,2
1,TRAIN_0002,CT007,기술,Series A,3132.0,Yes,Yes,54.0,6453.0,12141.0,4.0,3500-4500,0.5,7
2,TRAIN_0004,CT002,에듀테크,Seed,1969.0,No,Yes,94.0,829.0,9810.0,1.0,1500-2500,0.1,5
3,TRAIN_0005,CT008,기술,Series C,3801.0,No,Yes,69.0,6505.0,12722.0,3.0,2500-3500,0.6,13
4,TRAIN_0007,CT002,게임,Series A,2617.0,No,No,80.0,5579.0,5399.0,4.0,3500-4500,0.7,5


In [ ]:
# ✅ '기업나이' 컬럼 생성 (현재년도 - 설립연도)
test_df_imputed['기업나이'] = current_year - test_df_imputed['설립연도']

# ✅ 기존 '설립연도' 컬럼 제거
test_df_imputed = test_df_imputed.drop(columns=['설립연도'])

In [ ]:
test_df_imputed.head()

,ID,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),기업나이
0,TEST_0000,CT010,핀테크,Series C,3261.0,No,Yes,45.0,5021.0,6680.0,2.00,1500-2500,23
1,TEST_0001,CT001,푸드테크,Series C,3707.0,Yes,No,70.0,1601.0,4654.0,4.20,4500-6000,5
2,TEST_0002,CT006,에듀테크,IPO,236.0,Yes,Yes,89.0,4709.0,9289.0,1.00,6000이상,11
3,TEST_0003,CT001,에너지,Seed,637.0,Yes,Yes,17.0,2145.0,7005.0,5.00,1500-2500,22
4,TEST_0004,CT010,핀테크,Seed,4922.0,Yes,No,68.0,4995.0,7593.0,4.36,4500-6000,19


---





In [ ]:
# ✅ 구간을 대표값으로 매핑할 딕셔너리
value_mapping = {
    '1500-2500': 2000,
    '2500-3500': 3000,
    '3500-4500': 4000,
    '4500-6000': 5250,
    '6000이상': 6500
}

# ✅ '기업가치(추정값)' 컬럼 생성
train_df_imputed['기업가치(추정값)'] = train_df_imputed['기업가치(백억원)'].map(value_mapping)


In [ ]:
train_df_imputed.head()

,ID,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),성공확률,기업나이,기업가치(추정값)
0,TRAIN_0001,CT006,핀테크,Seed,4167.0,Yes,No,80.0,4069.0,279.0,1.0,2500-3500,0.8,2,3000
1,TRAIN_0002,CT007,기술,Series A,3132.0,Yes,Yes,54.0,6453.0,12141.0,4.0,3500-4500,0.5,7,4000
2,TRAIN_0004,CT002,에듀테크,Seed,1969.0,No,Yes,94.0,829.0,9810.0,1.0,1500-2500,0.1,5,2000
3,TRAIN_0005,CT008,기술,Series C,3801.0,No,Yes,69.0,6505.0,12722.0,3.0,2500-3500,0.6,13,3000
4,TRAIN_0007,CT002,게임,Series A,2617.0,No,No,80.0,5579.0,5399.0,4.0,3500-4500,0.7,5,4000


In [ ]:
# ✅ 구간을 대표값으로 매핑할 딕셔너리
value_mapping = {
    '1500-2500': 2000,
    '2500-3500': 3000,
    '3500-4500': 4000,
    '4500-6000': 5250,
    '6000이상': 6500
}

# ✅ '기업가치(추정값)' 컬럼 생성
test_df_imputed['기업가치(추정값)'] = test_df_imputed['기업가치(백억원)'].map(value_mapping)

In [ ]:
test_df_imputed.head()

,ID,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(백억원),기업나이,기업가치(추정값)
0,TEST_0000,CT010,핀테크,Series C,3261.0,No,Yes,45.0,5021.0,6680.0,2.00,1500-2500,23,2000
1,TEST_0001,CT001,푸드테크,Series C,3707.0,Yes,No,70.0,1601.0,4654.0,4.20,4500-6000,5,5250
2,TEST_0002,CT006,에듀테크,IPO,236.0,Yes,Yes,89.0,4709.0,9289.0,1.00,6000이상,11,6500
3,TEST_0003,CT001,에너지,Seed,637.0,Yes,Yes,17.0,2145.0,7005.0,5.00,1500-2500,22,2000
4,TEST_0004,CT010,핀테크,Seed,4922.0,Yes,No,68.0,4995.0,7593.0,4.36,4500-6000,19,5250


---

# Feature Engineering

## 1. 범주형 - 범주형

### 2-gram

In [ ]:
# ✅ 결합할 컬럼쌍 리스트
comb_cols = [
    ('국가', '분야'),
    ('국가', '투자단계'),
    ('국가', '인수여부'),
    ('국가', '상장여부'),
    ('국가', '기업가치(백억원)'),
    ('분야', '투자단계'),
    ('분야', '인수여부'),
    ('분야', '상장여부'),
    ('분야', '기업가치(백억원)'),
    ('투자단계', '인수여부'),
    ('투자단계', '상장여부'),
    ('투자단계', '기업가치(백억원)'),
    ('인수여부', '상장여부'),
    ('인수여부', '기업가치(백억원)'),
    ('상장여부', '기업가치(백억원)')
]

# ✅ train 데이터에 조합 컬럼 생성
for col1, col2 in comb_cols:
    new_col = f'{col1}_{col2}'
    train_df_imputed[new_col] = train_df_imputed[col1].astype(str) + '_' + train_df_imputed[col2].astype(str)

# ✅ test 데이터에도 동일하게 조합 컬럼 생성
for col1, col2 in comb_cols:
    new_col = f'{col1}_{col2}'
    test_df_imputed[new_col] = test_df_imputed[col1].astype(str) + '_' + test_df_imputed[col2].astype(str)


### 3-gram

In [ ]:
# ✅ 3-gram 조합을 위한 컬럼 리스트
columns_3gram = ['국가', '분야', '투자단계', '인수여부', '상장여부', '기업가치(백억원)']

# ✅ 조합 가능한 모든 3-gram 컬럼쌍 추출
from itertools import combinations

triplet_combinations = list(combinations(columns_3gram, 3))

# ✅ train 데이터에 3-gram 조합 컬럼 생성
for col1, col2, col3 in triplet_combinations:
    new_col = f'{col1}_{col2}_{col3}'
    train_df_imputed[new_col] = (
        train_df_imputed[col1].astype(str) + '_' +
        train_df_imputed[col2].astype(str) + '_' +
        train_df_imputed[col3].astype(str)
    )

# ✅ test 데이터에 동일한 3-gram 조합 컬럼 생성
for col1, col2, col3 in triplet_combinations:
    new_col = f'{col1}_{col2}_{col3}'
    test_df_imputed[new_col] = (
        test_df_imputed[col1].astype(str) + '_' +
        test_df_imputed[col2].astype(str) + '_' +
        test_df_imputed[col3].astype(str)
    )



### 4-gram

In [ ]:
from itertools import combinations

# ✅ 사용할 컬럼들
base_columns = ['국가', '분야', '투자단계', '인수여부', '상장여부', '기업가치(백억원)']

# ✅ 4-gram 조합 만들기
quadruplet_combinations = list(combinations(base_columns, 4))

# ✅ train 데이터에 4-gram 생성
for cols in quadruplet_combinations:
    new_col = '_'.join(cols)
    train_df_imputed[new_col] = (
        train_df_imputed[cols[0]].astype(str) + '_' +
        train_df_imputed[cols[1]].astype(str) + '_' +
        train_df_imputed[cols[2]].astype(str) + '_' +
        train_df_imputed[cols[3]].astype(str)
    )

# ✅ test 데이터에도 동일한 4-gram 생성
for cols in quadruplet_combinations:
    new_col = '_'.join(cols)
    test_df_imputed[new_col] = (
        test_df_imputed[cols[0]].astype(str) + '_' +
        test_df_imputed[cols[1]].astype(str) + '_' +
        test_df_imputed[cols[2]].astype(str) + '_' +
        test_df_imputed[cols[3]].astype(str)
    )



### 5-gram

In [ ]:
from itertools import combinations

# ✅ 사용할 컬럼 목록
base_columns = ['국가', '분야', '투자단계', '인수여부', '상장여부', '기업가치(백억원)']

# ✅ 5-gram 조합 생성
quintuplet_combinations = list(combinations(base_columns, 5))

# ✅ train 데이터셋에 5-gram 파생 컬럼 생성
for cols in quintuplet_combinations:
    new_col = '_'.join(cols)
    train_df_imputed[new_col] = (
        train_df_imputed[cols[0]].astype(str) + '_' +
        train_df_imputed[cols[1]].astype(str) + '_' +
        train_df_imputed[cols[2]].astype(str) + '_' +
        train_df_imputed[cols[3]].astype(str) + '_' +
        train_df_imputed[cols[4]].astype(str)
    )

# ✅ test 데이터셋에도 동일한 5-gram 파생 컬럼 생성
for cols in quintuplet_combinations:
    new_col = '_'.join(cols)
    test_df_imputed[new_col] = (
        test_df_imputed[cols[0]].astype(str) + '_' +
        test_df_imputed[cols[1]].astype(str) + '_' +
        test_df_imputed[cols[2]].astype(str) + '_' +
        test_df_imputed[cols[3]].astype(str) + '_' +
        test_df_imputed[cols[4]].astype(str)
    )


### 6-gram

In [ ]:
from itertools import combinations

# ✅ 사용할 전체 컬럼
base_columns = ['국가', '분야', '투자단계', '인수여부', '상장여부', '기업가치(백억원)']

# ✅ 6-gram은 전체 조합 하나뿐
cols = base_columns
new_col = '_'.join(cols)

# ✅ train 데이터에 6-gram 생성
train_df_imputed[new_col] = (
    train_df_imputed[cols[0]].astype(str) + '_' +
    train_df_imputed[cols[1]].astype(str) + '_' +
    train_df_imputed[cols[2]].astype(str) + '_' +
    train_df_imputed[cols[3]].astype(str) + '_' +
    train_df_imputed[cols[4]].astype(str) + '_' +
    train_df_imputed[cols[5]].astype(str)
)

# ✅ test 데이터에도 동일한 6-gram 생성
test_df_imputed[new_col] = (
    test_df_imputed[cols[0]].astype(str) + '_' +
    test_df_imputed[cols[1]].astype(str) + '_' +
    test_df_imputed[cols[2]].astype(str) + '_' +
    test_df_imputed[cols[3]].astype(str) + '_' +
    test_df_imputed[cols[4]].astype(str) + '_' +
    test_df_imputed[cols[5]].astype(str)
)


In [ ]:
train_df_imputed.head()

,ID,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),...,분야_투자단계_상장여부_기업가치(백억원),분야_인수여부_상장여부_기업가치(백억원),투자단계_인수여부_상장여부_기업가치(백억원),국가_분야_투자단계_인수여부_상장여부,국가_분야_투자단계_인수여부_기업가치(백억원),국가_분야_투자단계_상장여부_기업가치(백억원),국가_분야_인수여부_상장여부_기업가치(백억원),국가_투자단계_인수여부_상장여부_기업가치(백억원),분야_투자단계_인수여부_상장여부_기업가치(백억원),국가_분야_투자단계_인수여부_상장여부_기업가치(백억원)
0,TRAIN_0001,CT006,핀테크,Seed,4167.0,Yes,No,80.0,4069.0,279.0,...,핀테크_Seed_No_2500-3500,핀테크_Yes_No_2500-3500,Seed_Yes_No_2500-3500,CT006_핀테크_Seed_Yes_No,CT006_핀테크_Seed_Yes_2500-3500,CT006_핀테크_Seed_No_2500-3500,CT006_핀테크_Yes_No_2500-3500,CT006_Seed_Yes_No_2500-3500,핀테크_Seed_Yes_No_2500-3500,CT006_핀테크_Seed_Yes_No_2500-3500
1,TRAIN_0002,CT007,기술,Series A,3132.0,Yes,Yes,54.0,6453.0,12141.0,...,기술_Series A_Yes_3500-4500,기술_Yes_Yes_3500-4500,Series A_Yes_Yes_3500-4500,CT007_기술_Series A_Yes_Yes,CT007_기술_Series A_Yes_3500-4500,CT007_기술_Series A_Yes_3500-4500,CT007_기술_Yes_Yes_3500-4500,CT007_Series A_Yes_Yes_3500-4500,기술_Series A_Yes_Yes_3500-4500,CT007_기술_Series A_Yes_Yes_3500-4500
2,TRAIN_0004,CT002,에듀테크,Seed,1969.0,No,Yes,94.0,829.0,9810.0,...,에듀테크_Seed_Yes_1500-2500,에듀테크_No_Yes_1500-2500,Seed_No_Yes_1500-2500,CT002_에듀테크_Seed_No_Yes,CT002_에듀테크_Seed_No_1500-2500,CT002_에듀테크_Seed_Yes_1500-2500,CT002_에듀테크_No_Yes_1500-2500,CT002_Seed_No_Yes_1500-2500,에듀테크_Seed_No_Yes_1500-2500,CT002_에듀테크_Seed_No_Yes_1500-2500
3,TRAIN_0005,CT008,기술,Series C,3801.0,No,Yes,69.0,6505.0,12722.0,...,기술_Series C_Yes_2500-3500,기술_No_Yes_2500-3500,Series C_No_Yes_2500-3500,CT008_기술_Series C_No_Yes,CT008_기술_Series C_No_2500-3500,CT008_기술_Series C_Yes_2500-3500,CT008_기술_No_Yes_2500-3500,CT008_Series C_No_Yes_2500-3500,기술_Series C_No_Yes_2500-3500,CT008_기술_Series C_No_Yes_2500-3500
4,TRAIN_0007,CT002,게임,Series A,2617.0,No,No,80.0,5579.0,5399.0,...,게임_Series A_No_3500-4500,게임_No_No_3500-4500,Series A_No_No_3500-4500,CT002_게임_Series A_No_No,CT002_게임_Series A_No_3500-4500,CT002_게임_Series A_No_3500-4500,CT002_게임_No_No_3500-4500,CT002_Series A_No_No_3500-4500,게임_Series A_No_No_3500-4500,CT002_게임_Series A_No_No_3500-4500


In [ ]:
test_df_imputed.head()

,ID,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),...,분야_투자단계_상장여부_기업가치(백억원),분야_인수여부_상장여부_기업가치(백억원),투자단계_인수여부_상장여부_기업가치(백억원),국가_분야_투자단계_인수여부_상장여부,국가_분야_투자단계_인수여부_기업가치(백억원),국가_분야_투자단계_상장여부_기업가치(백억원),국가_분야_인수여부_상장여부_기업가치(백억원),국가_투자단계_인수여부_상장여부_기업가치(백억원),분야_투자단계_인수여부_상장여부_기업가치(백억원),국가_분야_투자단계_인수여부_상장여부_기업가치(백억원)
0,TEST_0000,CT010,핀테크,Series C,3261.0,No,Yes,45.0,5021.0,6680.0,...,핀테크_Series C_Yes_1500-2500,핀테크_No_Yes_1500-2500,Series C_No_Yes_1500-2500,CT010_핀테크_Series C_No_Yes,CT010_핀테크_Series C_No_1500-2500,CT010_핀테크_Series C_Yes_1500-2500,CT010_핀테크_No_Yes_1500-2500,CT010_Series C_No_Yes_1500-2500,핀테크_Series C_No_Yes_1500-2500,CT010_핀테크_Series C_No_Yes_1500-2500
1,TEST_0001,CT001,푸드테크,Series C,3707.0,Yes,No,70.0,1601.0,4654.0,...,푸드테크_Series C_No_4500-6000,푸드테크_Yes_No_4500-6000,Series C_Yes_No_4500-6000,CT001_푸드테크_Series C_Yes_No,CT001_푸드테크_Series C_Yes_4500-6000,CT001_푸드테크_Series C_No_4500-6000,CT001_푸드테크_Yes_No_4500-6000,CT001_Series C_Yes_No_4500-6000,푸드테크_Series C_Yes_No_4500-6000,CT001_푸드테크_Series C_Yes_No_4500-6000
2,TEST_0002,CT006,에듀테크,IPO,236.0,Yes,Yes,89.0,4709.0,9289.0,...,에듀테크_IPO_Yes_6000이상,에듀테크_Yes_Yes_6000이상,IPO_Yes_Yes_6000이상,CT006_에듀테크_IPO_Yes_Yes,CT006_에듀테크_IPO_Yes_6000이상,CT006_에듀테크_IPO_Yes_6000이상,CT006_에듀테크_Yes_Yes_6000이상,CT006_IPO_Yes_Yes_6000이상,에듀테크_IPO_Yes_Yes_6000이상,CT006_에듀테크_IPO_Yes_Yes_6000이상
3,TEST_0003,CT001,에너지,Seed,637.0,Yes,Yes,17.0,2145.0,7005.0,...,에너지_Seed_Yes_1500-2500,에너지_Yes_Yes_1500-2500,Seed_Yes_Yes_1500-2500,CT001_에너지_Seed_Yes_Yes,CT001_에너지_Seed_Yes_1500-2500,CT001_에너지_Seed_Yes_1500-2500,CT001_에너지_Yes_Yes_1500-2500,CT001_Seed_Yes_Yes_1500-2500,에너지_Seed_Yes_Yes_1500-2500,CT001_에너지_Seed_Yes_Yes_1500-2500
4,TEST_0004,CT010,핀테크,Seed,4922.0,Yes,No,68.0,4995.0,7593.0,...,핀테크_Seed_No_4500-6000,핀테크_Yes_No_4500-6000,Seed_Yes_No_4500-6000,CT010_핀테크_Seed_Yes_No,CT010_핀테크_Seed_Yes_4500-6000,CT010_핀테크_Seed_No_4500-6000,CT010_핀테크_Yes_No_4500-6000,CT010_Seed_Yes_No_4500-6000,핀테크_Seed_Yes_No_4500-6000,CT010_핀테크_Seed_Yes_No_4500-6000


## 2. 수치형 - 수치형

### 2-gram (곱셈)

In [ ]:
import itertools
import pandas as pd

# ✅ 변수명 매핑
cols = {
    '직원수': '직원 수',
    '고객수': '고객수(백만명)',
    '총투자금': '총 투자금(억원)',
    '연매출': '연매출(억원)',
    'SNS': 'SNS 팔로워 수(백만명)',
    '기업나이': '기업나이',
    '기업가치': '기업가치(추정값)'
}

# ✅ 2개씩 조합
pairs = list(itertools.combinations(cols.keys(), 2))

# ✅ 파생 변수 저장용 딕셔너리
train_features = {}
test_features = {}

# ✅ 파생 변수 생성
for a, b in pairs:
    col_a, col_b = cols[a], cols[b]
    new_col_name = f"{a}_곱_{b}"

    train_features[new_col_name] = train_df_imputed[col_a] * train_df_imputed[col_b]
    test_features[new_col_name]  = test_df_imputed[col_a]  * test_df_imputed[col_b]

# ✅ concat을 이용해 병합
train_df_imputed = pd.concat([train_df_imputed, pd.DataFrame(train_features)], axis=1)
test_df_imputed  = pd.concat([test_df_imputed,  pd.DataFrame(test_features)],  axis=1)



### 2-gram (비율, 나눗셈)

In [ ]:
import pandas as pd
import numpy as np
import itertools

# ✅ 실제 컬럼명 매핑
cols = {
    '직원수': '직원 수',
    '고객수': '고객수(백만명)',
    '총투자금': '총 투자금(억원)',
    '연매출': '연매출(억원)',
    'SNS': 'SNS 팔로워 수(백만명)',
    '기업나이': '기업나이',
    '기업가치': '기업가치(추정값)'
}

# ✅ 모든 2개 조합
pairs = list(itertools.combinations(cols.items(), 2))

# ✅ 파생변수 저장할 딕셔너리
train_features = {}
test_features = {}

# ✅ 파생 변수 계산
for (_, col1), (_, col2) in pairs:
    name1 = f"{col1}_div_{col2}"
    name2 = f"{col2}_div_{col1}"

    train_features[name1] = train_df_imputed[col1] / train_df_imputed[col2].replace(0, np.nan)
    train_features[name2] = train_df_imputed[col2] / train_df_imputed[col1].replace(0, np.nan)

    test_features[name1] = test_df_imputed[col1] / test_df_imputed[col2].replace(0, np.nan)
    test_features[name2] = test_df_imputed[col2] / test_df_imputed[col1].replace(0, np.nan)

# ✅ DataFrame으로 한 번에 추가
train_df_imputed = pd.concat([train_df_imputed, pd.DataFrame(train_features)], axis=1)
test_df_imputed = pd.concat([test_df_imputed, pd.DataFrame(test_features)], axis=1)



### 3-gram (곱셈)

In [ ]:
import itertools
import pandas as pd

# ✅ 변수명 간편 저장 (표기명 → 실제 컬럼명)
cols = {
    '직원수': '직원 수',
    '고객수': '고객수(백만명)',
    '총투자금': '총 투자금(억원)',
    '연매출': '연매출(억원)',
    'SNS': 'SNS 팔로워 수(백만명)',
    '기업나이': '기업나이',
    '기업가치': '기업가치(추정값)'
}

# ✅ 모든 3개 조합 생성
triplets = list(itertools.combinations(cols.items(), 3))

# ✅ 파생 변수 저장용 딕셔너리
train_features = {}
test_features = {}

# ✅ 파생 변수 생성 (train/test 동시에)
for (key1, col1), (key2, col2), (key3, col3) in triplets:
    new_col_name = f"{key1}_mul_{key2}_mul_{key3}"

    train_features[new_col_name] = (
        train_df_imputed[col1] * train_df_imputed[col2] * train_df_imputed[col3]
    )
    test_features[new_col_name] = (
        test_df_imputed[col1] * test_df_imputed[col2] * test_df_imputed[col3]
    )

# ✅ 한 번에 concat
train_df_imputed = pd.concat([train_df_imputed, pd.DataFrame(train_features)], axis=1)
test_df_imputed = pd.concat([test_df_imputed, pd.DataFrame(test_features)], axis=1)


### 3-gram (비율, 나눗셈)

In [ ]:
import numpy as np
from itertools import combinations
import pandas as pd

# ✅ 실제 컬럼명 딕셔너리
cols = {
    '직원수': '직원 수',
    '고객수': '고객수(백만명)',
    '총투자금': '총 투자금(억원)',
    '연매출': '연매출(억원)',
    'SNS': 'SNS 팔로워 수(백만명)',
    '기업나이': '기업나이',
    '기업가치': '기업가치(추정값)'
}

# ✅ 조합 생성
num_cols = list(cols.values())
triplets = list(combinations(num_cols, 3))

# ✅ 파생 변수 저장용 딕셔너리
train_features = {}
test_features = {}

# ✅ 반복하면서 각 조합으로 파생변수 생성
for col_a, col_b, col_c in triplets:
    # (A × B) / C
    name1 = f'{col_a}_mul_{col_b}_div_{col_c}'
    train_features[name1] = (train_df_imputed[col_a] * train_df_imputed[col_b]) / train_df_imputed[col_c].replace(0, np.nan)
    test_features[name1]  = (test_df_imputed[col_a]  * test_df_imputed[col_b])  / test_df_imputed[col_c].replace(0, np.nan)

    # (A × C) / B
    name2 = f'{col_a}_mul_{col_c}_div_{col_b}'
    train_features[name2] = (train_df_imputed[col_a] * train_df_imputed[col_c]) / train_df_imputed[col_b].replace(0, np.nan)
    test_features[name2]  = (test_df_imputed[col_a]  * test_df_imputed[col_c])  / test_df_imputed[col_b].replace(0, np.nan)

    # (B × C) / A
    name3 = f'{col_b}_mul_{col_c}_div_{col_a}'
    train_features[name3] = (train_df_imputed[col_b] * train_df_imputed[col_c]) / train_df_imputed[col_a].replace(0, np.nan)
    test_features[name3]  = (test_df_imputed[col_b]  * test_df_imputed[col_c])  / test_df_imputed[col_a].replace(0, np.nan)

# ✅ 한 번에 병합
train_df_imputed = pd.concat([train_df_imputed, pd.DataFrame(train_features)], axis=1)
test_df_imputed  = pd.concat([test_df_imputed,  pd.DataFrame(test_features)],  axis=1)


In [ ]:
train_df_imputed.head()

,ID,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),...,SNS 팔로워 수(백만명)_mul_기업나이_div_연매출(억원),연매출(억원)_mul_SNS 팔로워 수(백만명)_div_기업가치(추정값),연매출(억원)_mul_기업가치(추정값)_div_SNS 팔로워 수(백만명),SNS 팔로워 수(백만명)_mul_기업가치(추정값)_div_연매출(억원),연매출(억원)_mul_기업나이_div_기업가치(추정값),연매출(억원)_mul_기업가치(추정값)_div_기업나이,기업나이_mul_기업가치(추정값)_div_연매출(억원),SNS 팔로워 수(백만명)_mul_기업나이_div_기업가치(추정값),SNS 팔로워 수(백만명)_mul_기업가치(추정값)_div_기업나이,기업나이_mul_기업가치(추정값)_div_SNS 팔로워 수(백만명)
0,TRAIN_0001,CT006,핀테크,Seed,4167.0,Yes,No,80.0,4069.0,279.0,...,0.007168,0.093,837000.0,10.752688,0.186000,4.185000e+05,21.505376,0.000667,1500.000000,6000.0
1,TRAIN_0002,CT007,기술,Series A,3132.0,Yes,Yes,54.0,6453.0,12141.0,...,0.002306,12.141,12141000.0,1.317849,21.246750,6.937714e+06,2.306235,0.007000,2285.714286,7000.0
2,TRAIN_0004,CT002,에듀테크,Seed,1969.0,No,Yes,94.0,829.0,9810.0,...,0.000510,4.905,19620000.0,0.203874,24.525000,3.924000e+06,1.019368,0.002500,400.000000,10000.0
3,TRAIN_0005,CT008,기술,Series C,3801.0,No,Yes,69.0,6505.0,12722.0,...,0.003066,12.722,12722000.0,0.707436,55.128667,2.935846e+06,3.065556,0.013000,692.307692,13000.0
4,TRAIN_0007,CT002,게임,Series A,2617.0,No,No,80.0,5579.0,5399.0,...,0.003704,5.399,5399000.0,2.963512,6.748750,4.319200e+06,3.704390,0.005000,3200.000000,5000.0


In [ ]:
test_df_imputed.head()

,ID,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),...,SNS 팔로워 수(백만명)_mul_기업나이_div_연매출(억원),연매출(억원)_mul_SNS 팔로워 수(백만명)_div_기업가치(추정값),연매출(억원)_mul_기업가치(추정값)_div_SNS 팔로워 수(백만명),SNS 팔로워 수(백만명)_mul_기업가치(추정값)_div_연매출(억원),연매출(억원)_mul_기업나이_div_기업가치(추정값),연매출(억원)_mul_기업가치(추정값)_div_기업나이,기업나이_mul_기업가치(추정값)_div_연매출(억원),SNS 팔로워 수(백만명)_mul_기업나이_div_기업가치(추정값),SNS 팔로워 수(백만명)_mul_기업가치(추정값)_div_기업나이,기업나이_mul_기업가치(추정값)_div_SNS 팔로워 수(백만명)
0,TEST_0000,CT010,핀테크,Series C,3261.0,No,Yes,45.0,5021.0,6680.0,...,0.006886,6.680000,6.680000e+06,0.598802,76.820000,5.808696e+05,6.886228,0.023000,173.913043,23000.000000
1,TEST_0001,CT001,푸드테크,Series C,3707.0,Yes,No,70.0,1601.0,4654.0,...,0.004512,3.723200,5.817500e+06,4.737860,4.432381,4.886700e+06,5.640309,0.004000,4410.000000,6250.000000
2,TEST_0002,CT006,에듀테크,IPO,236.0,Yes,Yes,89.0,4709.0,9289.0,...,0.001184,1.429077,6.037850e+07,0.699752,15.719846,5.488955e+06,7.697276,0.001692,590.909091,71500.000000
3,TEST_0003,CT001,에너지,Seed,637.0,Yes,Yes,17.0,2145.0,7005.0,...,0.015703,17.512500,2.802000e+06,1.427552,77.055000,6.368182e+05,6.281228,0.055000,454.545455,8800.000000
4,TEST_0004,CT010,핀테크,Seed,4922.0,Yes,No,68.0,4995.0,7593.0,...,0.010910,6.305806,9.142947e+06,3.014619,27.479429,2.098066e+06,13.137100,0.015779,1204.736842,22878.440367


## 3. 수치형 - 범주형

### 범주별 집계

In [ ]:
import numpy as np
import pandas as pd

# ✅ 범주형 변수
categorical_cols = ['국가', '분야', '투자단계', '인수여부', '상장여부', '기업가치(백억원)']

# ✅ 수치형 변수
numeric_cols = ['직원 수', '고객수(백만명)', '총 투자금(억원)', '연매출(억원)',
                'SNS 팔로워 수(백만명)', '기업나이', '기업가치(추정값)']

# ✅ 사용할 통계량 (다양화됨)
agg_stats = ['mean', 'median', 'std', 'min', 'max', 'count', 'nunique', 'sum']

# ✅ 파생 변수 저장용 딕셔너리
train_features = {}
test_features = {}

# ✅ 통계 기반 파생 변수 생성
for cat_col in categorical_cols:
    for num_col in numeric_cols:
        grouped_stats = train_df_imputed.groupby(cat_col)[num_col].agg(agg_stats)

        for stat in agg_stats:
            new_feature = f'{cat_col}_별_{stat}_{num_col}'
            train_features[new_feature] = train_df_imputed[cat_col].map(grouped_stats[stat])
            test_features[new_feature]  = test_df_imputed[cat_col].map(grouped_stats[stat])

# ✅ concat으로 병합
train_df_imputed = pd.concat([train_df_imputed, pd.DataFrame(train_features)], axis=1)
test_df_imputed  = pd.concat([test_df_imputed,  pd.DataFrame(test_features)],  axis=1)


### 수치형 값의 비율 / 차이

In [ ]:
import pandas as pd

# ✅ 범주형 변수
categorical_cols = ['국가', '분야', '투자단계', '인수여부', '상장여부', '기업가치(백억원)']

# ✅ 수치형 변수
numeric_cols = ['직원 수', '고객수(백만명)', '총 투자금(억원)', '연매출(억원)',
                'SNS 팔로워 수(백만명)', '기업나이', '기업가치(추정값)']

# ✅ 파생 변수 저장용 딕셔너리
train_features = {}
test_features = {}

# ✅ 파생 변수 생성
for cat_col in categorical_cols:
    for num_col in numeric_cols:
        # ▶️ train 기준 groupby 평균, 표준편차
        group_mean = train_df_imputed.groupby(cat_col)[num_col].transform('mean')
        group_std  = train_df_imputed.groupby(cat_col)[num_col].transform('std')

        # ▶️ 파생 변수명 정의
        ratio_feature = f'{num_col}_대_{cat_col}_평균비율'
        diff_feature  = f'{num_col}_에서_{cat_col}_평균차감'
        zscore_feature = f'{num_col}_의_{cat_col}_zscore'

        # ▶️ train 데이터
        train_features[ratio_feature] = train_df_imputed[num_col] / group_mean
        train_features[diff_feature]  = train_df_imputed[num_col] - group_mean
        train_features[zscore_feature] = (train_df_imputed[num_col] - group_mean) / group_std.replace(0, 1)

        # ▶️ test 기준 평균/표준편차 매핑
        mean_map = train_df_imputed.groupby(cat_col)[num_col].mean()
        std_map  = train_df_imputed.groupby(cat_col)[num_col].std().replace(0, 1)

        test_features[ratio_feature] = test_df_imputed[num_col] / test_df_imputed[cat_col].map(mean_map)
        test_features[diff_feature]  = test_df_imputed[num_col] - test_df_imputed[cat_col].map(mean_map)
        test_features[zscore_feature] = (
            test_df_imputed[num_col] - test_df_imputed[cat_col].map(mean_map)
        ) / test_df_imputed[cat_col].map(std_map)

# ✅ concat으로 병합
train_df_imputed = pd.concat([train_df_imputed, pd.DataFrame(train_features)], axis=1)
test_df_imputed  = pd.concat([test_df_imputed,  pd.DataFrame(test_features)],  axis=1)


### 수치형 랭킹

In [ ]:
import pandas as pd

# ✅ 범주형 변수
categorical_cols = ['국가', '분야', '투자단계', '인수여부', '상장여부', '기업가치(백억원)']

# ✅ 수치형 변수
numeric_cols = ['직원 수', '고객수(백만명)', '총 투자금(억원)', '연매출(억원)',
                'SNS 팔로워 수(백만명)', '기업나이', '기업가치(추정값)']

# ✅ 파생 변수 저장용 딕셔너리
train_features = {}
test_features = {}

# ✅ 랭킹 파생 변수 생성
for cat_col in categorical_cols:
    for num_col in numeric_cols:
        new_feature = f'{num_col}_의_{cat_col}_내순위'

        # ✅ train: 그룹 내 순위 계산
        train_features[new_feature] = train_df_imputed.groupby(cat_col)[num_col].rank(method='average')

        # ✅ test: 기준이 될 group별 값 가져오기
        # → 각 그룹에서 수치형을 정렬하여 순위를 매기는 함수 정의
        rank_map = (
            train_df_imputed.groupby(cat_col)[num_col]
            .rank(method='average')
            .groupby(train_df_imputed[cat_col])
            .apply(lambda x: dict(zip(train_df_imputed.loc[x.index, num_col], x)))
        )

        # 테스트 데이터에 순위 매핑
        def get_rank(val, group):
            group_map = rank_map.get(group, {})
            return group_map.get(val, np.nan)

        test_features[new_feature] = test_df_imputed.apply(
            lambda row: get_rank(row[num_col], row[cat_col]), axis=1
        )

# ✅ 병합
train_df_imputed = pd.concat([train_df_imputed, pd.DataFrame(train_features)], axis=1)
test_df_imputed  = pd.concat([test_df_imputed,  pd.DataFrame(test_features)],  axis=1)


### 범주별 수치 범주화 (Binning + 조합)

In [ ]:
# import pandas as pd

# # ✅ 범주형 변수
# categorical_cols = ['국가', '분야', '투자단계', '인수여부', '상장여부', '기업가치(백억원)']

# # ✅ 수치형 변수
# numeric_cols = ['직원 수', '고객수(백만명)', '총 투자금(억원)', '연매출(억원)',
#                 'SNS 팔로워 수(백만명)', '기업나이', '기업가치(추정값)']

# # ✅ 구간 개수 설정
# n_bins = 4

# # ✅ 파생 변수 저장용 딕셔너리
# train_features = {}
# test_features = {}

# # ✅ 파생 변수 생성
# for cat_col in categorical_cols:
#     for num_col in numeric_cols:
#         # ✅ 구간화 (train 기준 qcut 사용)
#         bin_col_name = f'{num_col}_bin'
#         bin_labels = [f'Q{i}' for i in range(n_bins)]
#         train_bins = pd.qcut(train_df_imputed[num_col], q=n_bins, labels=bin_labels, duplicates='drop')
#         test_bins = pd.cut(test_df_imputed[num_col], bins=pd.qcut(train_df_imputed[num_col], q=n_bins, duplicates='drop').categories, labels=bin_labels, include_lowest=True)

#         # ✅ 조합 변수 생성
#         new_feature = f'{cat_col}_X_{num_col}_bin'
#         train_features[new_feature] = train_df_imputed[cat_col].astype(str) + "_" + train_bins.astype(str)
#         test_features[new_feature]  = test_df_imputed[cat_col].astype(str) + "_" + test_bins.astype(str)

# # ✅ 병합
# train_df_imputed = pd.concat([train_df_imputed, pd.DataFrame(train_features)], axis=1)
# test_df_imputed  = pd.concat([test_df_imputed,  pd.DataFrame(test_features)],  axis=1)


In [ ]:
train_df_imputed.head()

,ID,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),...,SNS 팔로워 수(백만명)_의_상장여부_내순위,기업나이_의_상장여부_내순위,기업가치(추정값)_의_상장여부_내순위,직원 수_의_기업가치(백억원)_내순위,고객수(백만명)_의_기업가치(백억원)_내순위,총 투자금(억원)_의_기업가치(백억원)_내순위,연매출(억원)_의_기업가치(백억원)_내순위,SNS 팔로워 수(백만명)_의_기업가치(백억원)_내순위,기업나이_의_기업가치(백억원)_내순위,기업가치(추정값)_의_기업가치(백억원)_내순위
0,TRAIN_0001,CT006,핀테크,Seed,4167.0,Yes,No,80.0,4069.0,279.0,...,230.5,24.5,393.5,428.0,409.0,299.0,6.0,83.0,14.0,254.5
1,TRAIN_0002,CT007,기술,Series A,3132.0,Yes,Yes,54.0,6453.0,12141.0,...,1001.0,275.0,603.5,314.0,255.5,484.5,447.0,398.0,120.0,246.0
2,TRAIN_0004,CT002,에듀테크,Seed,1969.0,No,Yes,94.0,829.0,9810.0,...,190.5,187.5,119.5,200.0,479.0,61.0,384.0,77.0,61.0,250.5
3,TRAIN_0005,CT008,기술,Series C,3801.0,No,Yes,69.0,6505.0,12722.0,...,716.0,608.5,361.5,395.0,357.5,501.0,488.0,285.0,264.5,254.5
4,TRAIN_0007,CT002,게임,Series A,2617.0,No,No,80.0,5579.0,5399.0,...,1049.5,193.5,651.0,274.0,414.0,417.0,191.0,398.0,75.0,246.0


In [ ]:
test_df_imputed.head()

,ID,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),...,SNS 팔로워 수(백만명)_의_상장여부_내순위,기업나이_의_상장여부_내순위,기업가치(추정값)_의_상장여부_내순위,직원 수_의_기업가치(백억원)_내순위,고객수(백만명)_의_기업가치(백억원)_내순위,총 투자금(억원)_의_기업가치(백억원)_내순위,연매출(억원)_의_기업가치(백억원)_내순위,SNS 팔로워 수(백만명)_의_기업가치(백억원)_내순위,기업나이_의_기업가치(백억원)_내순위,기업가치(추정값)_의_기업가치(백억원)_내순위
0,TEST_0000,CT010,핀테크,Series C,3261.0,No,Yes,45.0,5021.0,6680.0,...,433.5,1161.0,119.5,NaN,234.0,NaN,NaN,182.5,473.0,250.5
1,TEST_0001,CT001,푸드테크,Series C,3707.0,Yes,No,70.0,1601.0,4654.0,...,NaN,193.5,907.5,372.0,368.5,NaN,NaN,NaN,79.0,251.5
2,TEST_0002,CT006,에듀테크,IPO,236.0,Yes,Yes,89.0,4709.0,9289.0,...,190.5,492.5,1101.0,NaN,505.0,NaN,NaN,79.0,218.0,264.0
3,TEST_0003,CT001,에너지,Seed,637.0,Yes,Yes,17.0,2145.0,7005.0,...,1188.5,1102.0,119.5,NaN,78.5,NaN,NaN,480.0,450.5,250.5
4,TEST_0004,CT010,핀테크,Seed,4922.0,Yes,No,68.0,4995.0,7593.0,...,NaN,997.0,907.5,NaN,362.0,NaN,NaN,NaN,385.5,251.5


---

# 범주형 변수 Encoding

## 1. Label Encoding

In [ ]:
# ✅ ID 컬럼 제외한 범주형 변수 추출
categorical_cols = [
    col for col in train_df_imputed.select_dtypes(include=["object", "category"]).columns
    if col.lower() != 'id'
]

# ✅ 파생 변수 저장용 딕셔너리
train_features = {}
test_features = {}

# ✅ Label Encoding 수행
for col in categorical_cols:
    # 범주형으로 변환
    train_df_imputed[col] = train_df_imputed[col].astype("category")
    test_df_imputed[col] = test_df_imputed[col].astype("category")

    # 카테고리 객체의 category 목록
    categories = train_df_imputed[col].cat.categories

    # 파생 변수 이름
    new_col = f'LE_{col}'

    # train: 직접 코드로 변환
    train_features[new_col] = train_df_imputed[col].cat.codes

    # test: train의 categories 기준으로 인덱스 매핑
    test_features[new_col] = test_df_imputed[col].apply(
        lambda x: categories.get_loc(x) if x in categories else -1
    )

# ✅ 한 번에 병합
train_df_imputed = pd.concat([train_df_imputed, pd.DataFrame(train_features)], axis=1)
test_df_imputed  = pd.concat([test_df_imputed,  pd.DataFrame(test_features)],  axis=1)


## 2. Count Encoding

In [ ]:
# ✅ ID 제외한 범주형 변수 추출
categorical_cols = [
    col for col in train_df_imputed.select_dtypes(include=["object", "category"]).columns
    if col.lower() != 'id'
]

# ✅ 파생 변수 저장용 딕셔너리
train_features = {}
test_features = {}

# ✅ Count Encoding 수행
for col in categorical_cols:
    # train에서 value_counts 기반 생성
    value_counts = train_df_imputed[col].value_counts()

    # 파생 변수 이름
    new_col = f'CE_{col}'

    # 각 데이터셋에 매핑 결과 저장
    train_features[new_col] = train_df_imputed[col].map(value_counts)
    test_features[new_col]  = test_df_imputed[col].map(value_counts)

# ✅ 한 번에 병합
train_df_imputed = pd.concat([train_df_imputed, pd.DataFrame(train_features)], axis=1)
test_df_imputed  = pd.concat([test_df_imputed,  pd.DataFrame(test_features)],  axis=1)


## 3. Target Encoding (mean, median, min, max, nunique)

In [ ]:
import numpy as np
import pandas as pd

# ✅ 복사본 생성
train = train_df_imputed.copy()
valid = train_df_imputed.copy()
test = test_df_imputed.copy()

# ✅ 타깃 변수
target_col = "성공확률"

# ✅ 사용할 집계 함수
agg_types = ["mean", "median", "min", "max", "nunique"]

# ✅ 범주형 변수 (ID 제외)
categorical_cols = [
    col for col in train.select_dtypes(include=["object", "category"]).columns
    if col.lower() != 'id'
]

# ✅ 파생 변수 저장용 딕셔너리
train_features = {}
valid_features = {}
test_features = {}

# ✅ 타깃 인코딩 함수 (1컬럼 1통계)
def target_encode_single(train_df, valid_df, test_df, col, target="성공확률", kfold=5, smooth=20, agg="mean"):
    train_df = train_df.copy()
    train_df["kfold"] = train_df.index % kfold
    col_name = '_'.join(col)
    enc_col = f'TE_{agg.upper()}_{col_name}'
    encoded = pd.Series(index=train_df.index, dtype="float32")

    for i in range(kfold):
        df_train = train_df[train_df["kfold"] != i]
        df_val = train_df[train_df["kfold"] == i]

        mn = getattr(df_train[target], agg)() if agg != "nunique" else 0
        df_agg = df_train[col + [target]].groupby(col).agg([agg, 'count']).reset_index()
        df_agg.columns = col + [agg, 'count']

        if agg == "nunique":
            df_agg["TE_tmp"] = df_agg[agg] / df_agg["count"]
        else:
            df_agg["TE_tmp"] = ((df_agg[agg] * df_agg["count"]) + (mn * smooth)) / (df_agg["count"] + smooth)

        df_val = df_val.merge(df_agg[col + ["TE_tmp"]], on=col, how="left")
        encoded.loc[df_val.index] = df_val["TE_tmp"].fillna(mn).astype("float32")

    # 최종 통계로 valid/test 생성
    df_final = train_df[col + [target]].groupby(col).agg([agg, 'count']).reset_index()
    df_final.columns = col + [agg, 'count']
    mn = getattr(train_df[target], agg)() if agg != "nunique" else 0

    if agg == "nunique":
        df_final["TE_tmp"] = df_final[agg] / df_final["count"]
    else:
        df_final["TE_tmp"] = ((df_final[agg] * df_final["count"]) + (mn * smooth)) / (df_final["count"] + smooth)

    valid_encoded = valid_df[col].merge(df_final[col + ["TE_tmp"]], how="left", on=col)["TE_tmp"].fillna(mn).astype("float32")
    test_encoded = test_df[col].merge(df_final[col + ["TE_tmp"]], how="left", on=col)["TE_tmp"].fillna(mn).astype("float32")

    return enc_col, encoded, valid_encoded, test_encoded

# ✅ 모든 조합에 대해 인코딩 실행
for col in categorical_cols:
    for agg in agg_types:
        enc_col, train_enc, valid_enc, test_enc = target_encode_single(
            train, valid, test, [col], target=target_col, kfold=5, smooth=20, agg=agg
        )
        train_features[enc_col] = train_enc
        valid_features[enc_col] = valid_enc
        test_features[enc_col] = test_enc

# ✅ concat으로 병합
train_df_imputed = pd.concat([train, pd.DataFrame(train_features)], axis=1)
test_df_imputed  = pd.concat([test,  pd.DataFrame(test_features)],  axis=1)


<ipython-input-53-0502e6ed556b>:39: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_agg = df_train[col + [target]].groupby(col).agg([agg, 'count']).reset_index()
<ipython-input-53-0502e6ed556b>:39: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_agg = df_train[col + [target]].groupby(col).agg([agg, 'count']).reset_index()
<ipython-input-53-0502e6ed556b>:39: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_agg = df_trai

In [ ]:
train_df_imputed.head()

,ID,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),...,TE_MEAN_CE_투자단계_상장여부,TE_MEDIAN_CE_투자단계_상장여부,TE_MIN_CE_투자단계_상장여부,TE_MAX_CE_투자단계_상장여부,TE_NUNIQUE_CE_투자단계_상장여부,TE_MEAN_CE_인수여부_상장여부,TE_MEDIAN_CE_인수여부_상장여부,TE_MIN_CE_인수여부_상장여부,TE_MAX_CE_인수여부_상장여부,TE_NUNIQUE_CE_인수여부_상장여부
0,TRAIN_0001,CT006,핀테크,Seed,4167.0,Yes,No,80.0,4069.0,279.0,...,0.517005,0.500000,0.1,0.9,0.037559,0.531158,0.500000,0.1,0.9,0.017505
1,TRAIN_0002,CT007,기술,Series A,3132.0,Yes,Yes,54.0,6453.0,12141.0,...,0.547320,0.591597,0.1,0.9,0.036697,0.555159,0.596124,0.1,0.9,0.016129
2,TRAIN_0004,CT002,에듀테크,Seed,1969.0,No,Yes,94.0,829.0,9810.0,...,0.547320,0.591597,0.1,0.9,0.036697,0.553455,0.596078,0.1,0.9,0.016327
3,TRAIN_0005,CT008,기술,Series C,3801.0,No,Yes,69.0,6505.0,12722.0,...,0.517005,0.500000,0.1,0.9,0.037559,0.515937,0.500000,0.1,0.9,0.013793
4,TRAIN_0007,CT002,게임,Series A,2617.0,No,No,80.0,5579.0,5399.0,...,0.497814,0.500000,0.1,0.9,0.035398,0.531158,0.500000,0.1,0.9,0.017505


In [ ]:
test_df_imputed.head()

,ID,국가,분야,투자단계,직원 수,인수여부,상장여부,고객수(백만명),총 투자금(억원),연매출(억원),...,TE_MEAN_CE_투자단계_상장여부,TE_MEDIAN_CE_투자단계_상장여부,TE_MIN_CE_투자단계_상장여부,TE_MAX_CE_투자단계_상장여부,TE_NUNIQUE_CE_투자단계_상장여부,TE_MEAN_CE_인수여부_상장여부,TE_MEDIAN_CE_인수여부_상장여부,TE_MIN_CE_인수여부_상장여부,TE_MAX_CE_인수여부_상장여부,TE_NUNIQUE_CE_인수여부_상장여부
0,TEST_0000,CT010,핀테크,Series C,3261.0,No,Yes,45.0,5021.0,6680.0,...,0.533266,0.500000,0.1,0.9,0.030303,0.550386,0.596885,0.1,0.9,0.012862
1,TEST_0001,CT001,푸드테크,Series C,3707.0,Yes,No,70.0,1601.0,4654.0,...,0.524562,0.500000,0.1,0.9,0.037383,0.520365,0.500000,0.1,0.9,0.011204
2,TEST_0002,CT006,에듀테크,IPO,236.0,Yes,Yes,89.0,4709.0,9289.0,...,0.543931,0.593266,0.1,0.9,0.028881,0.549366,0.596850,0.1,0.9,0.013008
3,TEST_0003,CT001,에너지,Seed,637.0,Yes,Yes,17.0,2145.0,7005.0,...,0.570353,0.591935,0.1,0.9,0.035088,0.549366,0.596850,0.1,0.9,0.013008
4,TEST_0004,CT010,핀테크,Seed,4922.0,Yes,No,68.0,4995.0,7593.0,...,0.512574,0.500000,0.1,0.9,0.027972,0.520365,0.500000,0.1,0.9,0.011204


---

# 범주형 변수 제거

In [ ]:
# ✅ ID 제외한 범주형 변수 자동 추출
categorical_cols = [
    col for col in train_df_imputed.select_dtypes(include=["object", "category"]).columns
    if col.lower() != 'id'
]

# ✅ train, test에서 범주형 변수 제거
train_df_imputed = train_df_imputed.drop(columns=categorical_cols)
test_df_imputed = test_df_imputed.drop(columns=categorical_cols)


In [ ]:
train_df_imputed.head()

,ID,직원 수,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),성공확률,기업나이,기업가치(추정값),직원수_곱_고객수,...,TE_MEAN_CE_투자단계_상장여부,TE_MEDIAN_CE_투자단계_상장여부,TE_MIN_CE_투자단계_상장여부,TE_MAX_CE_투자단계_상장여부,TE_NUNIQUE_CE_투자단계_상장여부,TE_MEAN_CE_인수여부_상장여부,TE_MEDIAN_CE_인수여부_상장여부,TE_MIN_CE_인수여부_상장여부,TE_MAX_CE_인수여부_상장여부,TE_NUNIQUE_CE_인수여부_상장여부
0,TRAIN_0001,4167.0,80.0,4069.0,279.0,1.0,0.8,2,3000,333360.0,...,0.517005,0.500000,0.1,0.9,0.037559,0.531158,0.500000,0.1,0.9,0.017505
1,TRAIN_0002,3132.0,54.0,6453.0,12141.0,4.0,0.5,7,4000,169128.0,...,0.547320,0.591597,0.1,0.9,0.036697,0.555159,0.596124,0.1,0.9,0.016129
2,TRAIN_0004,1969.0,94.0,829.0,9810.0,1.0,0.1,5,2000,185086.0,...,0.547320,0.591597,0.1,0.9,0.036697,0.553455,0.596078,0.1,0.9,0.016327
3,TRAIN_0005,3801.0,69.0,6505.0,12722.0,3.0,0.6,13,3000,262269.0,...,0.517005,0.500000,0.1,0.9,0.037559,0.515937,0.500000,0.1,0.9,0.013793
4,TRAIN_0007,2617.0,80.0,5579.0,5399.0,4.0,0.7,5,4000,209360.0,...,0.497814,0.500000,0.1,0.9,0.035398,0.531158,0.500000,0.1,0.9,0.017505


In [ ]:
test_df_imputed.head()

,ID,직원 수,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업나이,기업가치(추정값),직원수_곱_고객수,직원수_곱_총투자금,...,TE_MEAN_CE_투자단계_상장여부,TE_MEDIAN_CE_투자단계_상장여부,TE_MIN_CE_투자단계_상장여부,TE_MAX_CE_투자단계_상장여부,TE_NUNIQUE_CE_투자단계_상장여부,TE_MEAN_CE_인수여부_상장여부,TE_MEDIAN_CE_인수여부_상장여부,TE_MIN_CE_인수여부_상장여부,TE_MAX_CE_인수여부_상장여부,TE_NUNIQUE_CE_인수여부_상장여부
0,TEST_0000,3261.0,45.0,5021.0,6680.0,2.00,23,2000,146745.0,16373481.0,...,0.533266,0.500000,0.1,0.9,0.030303,0.550386,0.596885,0.1,0.9,0.012862
1,TEST_0001,3707.0,70.0,1601.0,4654.0,4.20,5,5250,259490.0,5934907.0,...,0.524562,0.500000,0.1,0.9,0.037383,0.520365,0.500000,0.1,0.9,0.011204
2,TEST_0002,236.0,89.0,4709.0,9289.0,1.00,11,6500,21004.0,1111324.0,...,0.543931,0.593266,0.1,0.9,0.028881,0.549366,0.596850,0.1,0.9,0.013008
3,TEST_0003,637.0,17.0,2145.0,7005.0,5.00,22,2000,10829.0,1366365.0,...,0.570353,0.591935,0.1,0.9,0.035088,0.549366,0.596850,0.1,0.9,0.013008
4,TEST_0004,4922.0,68.0,4995.0,7593.0,4.36,19,5250,334696.0,24585390.0,...,0.512574,0.500000,0.1,0.9,0.027972,0.520365,0.500000,0.1,0.9,0.011204


---

In [ ]:
# from sklearn.model_selection import cross_val_score
# from sklearn.metrics import mean_absolute_error, make_scorer
# from lightgbm import LGBMRegressor

# # ✅ 설정
# target_col = '성공확률'

# # ✅ ID 제외한 피처 정의
# X = train_df_imputed.drop(columns=[target_col])
# X = X[[col for col in X.columns if col.lower() != 'id']]  # ID 컬럼 제거

# y = train_df_imputed[target_col]

# # ✅ GPU 환경용 pandas 변환 (cuDF 대응용)
# if hasattr(X, "to_pandas"):
#     X = X.to_pandas()
#     y = y.to_pandas()

# # ✅ 전진 선택용 변수 초기화
# all_features = X.columns.tolist()
# selected = []
# best_score = float('inf')

# # ✅ GPU용 LightGBM 모델 정의
# model = LGBMRegressor(
#     device="gpu",
#     boosting_type="gbdt",
#     n_estimators=100,
#     learning_rate=0.05,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1,
# )

# # ✅ 평가 함수 정의 (MAE, 낮을수록 좋음)
# scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# print("🔍 전진 선택 시작 (GPU 사용)\n")

# # ✅ 전진 선택 루프
# for col in all_features:
#     trial_features = selected + [col]
#     score = -cross_val_score(model, X[trial_features], y, scoring=scorer, cv=5).mean()

#     if score < best_score:
#         selected.append(col)
#         best_score = score
#         print(f"✅ 선택됨: {col:30} | MAE 하락: {score:.5f}")
#     else:
#         print(f"❌ 제외됨: {col:30} | MAE 상승: {score:.5f}")

# # ✅ 결과 출력
# print("\n🎯 최종 선택된 Features:")
# print(selected)
# print(f"📉 최종 MAE: {best_score:.5f}")


In [ ]:
# from sklearn.model_selection import cross_val_score
# from sklearn.metrics import mean_absolute_error, make_scorer
# from xgboost import XGBRegressor

# # ✅ 설정
# target_col = '성공확률'

# # ✅ ID 제외한 피처 정의
# X = train_df_imputed.drop(columns=[target_col])
# X = X[[col for col in X.columns if col.lower() != 'id']]  # ID 컬럼 제거

# y = train_df_imputed[target_col]

# # ✅ GPU 환경용 pandas 변환 (cuDF 대응용)
# if hasattr(X, "to_pandas"):
#     X = X.to_pandas()
#     y = y.to_pandas()

# # ✅ 전진 선택용 변수 초기화
# all_features = X.columns.tolist()
# selected = []
# best_score = float('inf')

# # ✅ GPU용 XGBoost 모델 정의
# model = XGBRegressor(
#     tree_method="gpu_hist",     # GPU 사용 설정
#     predictor="gpu_predictor",  # GPU 예측기
#     n_estimators=100,
#     learning_rate=0.05,
#     random_state=42,
#     n_jobs=-1,
#     verbosity=0
# )

# # ✅ 평가 함수 정의 (MAE, 낮을수록 좋음)
# scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# print("🔍 전진 선택 시작 (XGBoost + GPU 사용)\n")

# # ✅ 전진 선택 루프
# for col in all_features:
#     trial_features = selected + [col]
#     score = -cross_val_score(model, X[trial_features], y, scoring=scorer, cv=5).mean()

#     if score < best_score:
#         selected.append(col)
#         best_score = score
#         print(f"✅ 선택됨: {col:30} | MAE 하락: {score:.5f}")
#     else:
#         print(f"❌ 제외됨: {col:30} | MAE 상승: {score:.5f}")

# # ✅ 결과 출력
# print("\n🎯 최종 선택된 Features:")
# print(selected)
# print(f"📉 최종 MAE: {best_score:.5f}")


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

# ✅ 설정
target_col = '성공확률'

# ✅ ID 제외한 피처 정의
X = train_df_imputed.drop(columns=[target_col])
X = X[[col for col in X.columns if col.lower() != 'id']]  # ID 컬럼 제거

y = train_df_imputed[target_col]

# ✅ GPU 환경 대응 (cuDF 사용 시 to_pandas)
if hasattr(X, "to_pandas"):
    X = X.to_pandas()
    y = y.to_pandas()

# ✅ 전진 선택용 변수 초기화
all_features = X.columns.tolist()
selected = []
best_score = float('inf')

# ✅ RandomForest 모델 정의 (CPU 기반)
model = RandomForestRegressor()

# ✅ 평가 함수 정의 (MAE, 낮을수록 좋음)
scorer = make_scorer(mean_absolute_error, greater_is_better=False)

print("🔍 전진 선택 시작 (RandomForest 사용)\n")

# ✅ 전진 선택 루프
for col in all_features:
    trial_features = selected + [col]
    score = -cross_val_score(model, X[trial_features], y, scoring=scorer, cv=5).mean()

    if score < best_score:
        selected.append(col)
        best_score = score
        print(f"✅ 선택됨: {col:30} | MAE 하락: {score:.5f}")
    else:
        print(f"❌ 제외됨: {col:30} | MAE 상승: {score:.5f}")

# ✅ 결과 출력
print("\n🎯 최종 선택된 Features:")
print(selected)
print(f"📉 최종 MAE: {best_score:.5f}")


🔍 전진 선택 시작 (RandomForest 사용)

✅ 선택됨: 직원 수                           | MAE 하락: 0.22488
✅ 선택됨: 고객수(백만명)                       | MAE 하락: 0.20694
✅ 선택됨: 총 투자금(억원)                      | MAE 하락: 0.19190
✅ 선택됨: 연매출(억원)                        | MAE 하락: 0.18229
✅ 선택됨: SNS 팔로워 수(백만명)                 | MAE 하락: 0.18009
❌ 제외됨: 기업나이                           | MAE 상승: 0.18440
✅ 선택됨: 기업가치(추정값)                      | MAE 하락: 0.17722
❌ 제외됨: 직원수_곱_고객수                      | MAE 상승: 0.17931
❌ 제외됨: 직원수_곱_총투자금                     | MAE 상승: 0.17797
✅ 선택됨: 직원수_곱_연매출                      | MAE 하락: 0.17574
❌ 제외됨: 직원수_곱_SNS                      | MAE 상승: 0.17731
❌ 제외됨: 직원수_곱_기업나이                     | MAE 상승: 0.18105
❌ 제외됨: 직원수_곱_기업가치                     | MAE 상승: 0.17628
❌ 제외됨: 고객수_곱_총투자금                     | MAE 상승: 0.17766
❌ 제외됨: 고객수_곱_연매출                      | MAE 상승: 0.17780
❌ 제외됨: 고객수_곱_SNS                      | MAE 상승: 0.17762
❌ 제외됨: 고객수_곱_기업나이                     | MAE 상승: 0.18029
❌ 제외됨: 고객수_곱_기업가치 

KeyboardInterrupt: 

In [ ]:
fsdfsadvwewsdc

---

In [ ]:
# ✅ 남길 컬럼 리스트 정의 (train에는 '성공확률' 추가 포함)
selected_features = [
    '직원 수',
    '고객수(백만명)',
    '총 투자금(억원)',
    '연매출(억원)',
    'SNS 팔로워 수(백만명)',
    '기업가치(추정값)',
    '직원수_곱_연매출',
    '총투자금_곱_기업가치',
    '연매출_곱_기업가치',
    'SNS_곱_기업가치',
    '연매출(억원)_div_직원 수',
    '기업가치(추정값)_div_직원 수',
    '기업가치(추정값)_div_총 투자금(억원)',
    '연매출_mul_SNS_mul_기업가치',
    '직원 수_mul_기업가치(추정값)_div_연매출(억원)',
    '총 투자금(억원)_mul_연매출(억원)_div_고객수(백만명)'
]

# ✅ train에는 성공확률 추가
train_selected = train_df_imputed[selected_features + ['성공확률']].copy()

# ✅ test는 그대로
test_selected = test_df_imputed[selected_features].copy()




In [ ]:
train_selected.head()

,직원 수,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(추정값),직원수_곱_연매출,총투자금_곱_기업가치,연매출_곱_기업가치,SNS_곱_기업가치,연매출(억원)_div_직원 수,기업가치(추정값)_div_직원 수,기업가치(추정값)_div_총 투자금(억원),연매출_mul_SNS_mul_기업가치,직원 수_mul_기업가치(추정값)_div_연매출(억원),총 투자금(억원)_mul_연매출(억원)_div_고객수(백만명),성공확률
0,4167.0,80.0,4069.0,279.0,1.0,3000,1162593.0,12207000.0,837000.0,3000.0,0.066955,0.719942,0.737282,837000.0,44806.451613,1.419064e+04,0.8
1,3132.0,54.0,6453.0,12141.0,4.0,4000,38025612.0,25812000.0,48564000.0,16000.0,3.876437,1.277139,0.619867,194256000.0,1031.875463,1.450850e+06,0.5
2,1969.0,94.0,829.0,9810.0,1.0,2000,19315890.0,1658000.0,19620000.0,2000.0,4.982224,1.015744,2.412545,19620000.0,401.427115,8.651585e+04,0.1
3,3801.0,69.0,6505.0,12722.0,3.0,3000,48356322.0,19515000.0,38166000.0,9000.0,3.347014,0.789266,0.461184,114498000.0,896.321333,1.199371e+06,0.6
4,2617.0,80.0,5579.0,5399.0,4.0,4000,14129183.0,22316000.0,21596000.0,16000.0,2.063049,1.528468,0.716974,86384000.0,1938.877570,3.765128e+05,0.7


In [ ]:
test_selected.head()

,직원 수,고객수(백만명),총 투자금(억원),연매출(억원),SNS 팔로워 수(백만명),기업가치(추정값),직원수_곱_연매출,총투자금_곱_기업가치,연매출_곱_기업가치,SNS_곱_기업가치,연매출(억원)_div_직원 수,기업가치(추정값)_div_직원 수,기업가치(추정값)_div_총 투자금(억원),연매출_mul_SNS_mul_기업가치,직원 수_mul_기업가치(추정값)_div_연매출(억원),총 투자금(억원)_mul_연매출(억원)_div_고객수(백만명)
0,3261.0,45.0,5021.0,6680.0,2.00,2000,21783480.0,10042000.0,13360000.0,4000.0,2.048451,0.613309,0.398327,26720000.0,976.347305,745339.555556
1,3707.0,70.0,1601.0,4654.0,4.20,5250,17252378.0,8405250.0,24433500.0,22050.0,1.255463,1.416240,3.279200,102620700.0,4181.725398,106443.628571
2,236.0,89.0,4709.0,9289.0,1.00,6500,2192204.0,30608500.0,60378500.0,6500.0,39.360169,27.542373,1.380336,60378500.0,165.141565,491482.033708
3,637.0,17.0,2145.0,7005.0,5.00,2000,4462185.0,4290000.0,14010000.0,10000.0,10.996860,3.139717,0.932401,70050000.0,181.870093,883866.176471
4,4922.0,68.0,4995.0,7593.0,4.36,5250,37372746.0,26223750.0,39863250.0,22890.0,1.542666,1.066640,1.051051,173803770.0,3403.200316,557750.514706


---

In [ ]:
sample_sub.head()

,ID,성공확률
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0


In [ ]:
from xgboost import XGBRegressor
import pandas as pd

# ✅ 피처(X), 타깃(y) 분리
X_train = train_selected.drop(columns=['성공확률'])
y_train = train_selected['성공확률']

X_test = test_selected.copy()

# ✅ 모델 정의
model = XGBRegressor(
    tree_method="gpu_hist",     # GPU 사용 설정
    predictor="gpu_predictor",  # GPU 예측기
    n_estimators=100,
    learning_rate=0.05,
    random_state=42,
    n_jobs=-1,
    verbosity=0
)

# ✅ 학습
model.fit(X_train, y_train)

# ✅ 예측
test_preds = model.predict(X_test)

# ✅ sample_sub에 예측값 덮어쓰기
submission = sample_sub.copy()
submission['성공확률'] = test_preds

# ✅ 저장
submission.to_csv('submission.csv', index=False)
print("✅ submission.csv 파일이 저장되었습니다.")


In [ ]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

# ✅ 피처(X), 타깃(y) 분리
X_train = train_selected.drop(columns=['성공확률'])
y_train = train_selected['성공확률']

X_test = test_selected.copy()

# ✅ 모델 정의 (RandomForestRegressor)
model = RandomForestRegressor()

# ✅ 학습
model.fit(X_train, y_train)

# ✅ 예측
test_preds = model.predict(X_test)

# ✅ sample_sub에 예측값 덮어쓰기
submission = sample_sub.copy()
submission['성공확률'] = test_preds

# ✅ 저장
submission.to_csv('submission.csv', index=False)
print("✅ submission.csv 파일이 저장되었습니다.")


✅ submission.csv 파일이 저장되었습니다.
